### 1) One Hot Factorization Machine

In [4]:
class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dimensions, embedding_dimensions):
        
        super().__init__()
        
        fields_count = sum(field_dimensions)
        
        self.embedding = torch.nn.Embedding(fields_count, embedding_dimensions)
        
        field_offsets = np.cumsum(field_dimensions)
        
        self.embedding_offsets = np.array((0, *field_offsets[:-1]), dtype=int)
        
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)
        
        # print("fields_count:", fields_count)
        # print("field_offsets:", field_offsets)        
        # print("self.embedding_offsets:", self.embedding_offsets)
        # print("self.embedding.weight.data:", self.embedding.weight.data)
        
        
    def forward(self, embedding_inputs):
        
        one_hot_offsets = embedding_inputs.new_tensor(self.embedding_offsets).unsqueeze(0)

        one_hot_positions = embedding_inputs + one_hot_offsets
        
        embedding_weights = self.embedding(one_hot_positions)
        
        # print("embedding_inputs:", embedding_inputs)        
        # print("one_hot_positions:", one_hot_positions)        
        # print("embedding_weights:", embedding_weights)
        
        return embedding_weights
    
    
class FeaturesLinear(torch.nn.Module):

    def __init__(self, field_dims, output_dim=1):
        
        super().__init__()
        
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=int)

    def forward(self, x):

        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        
        return torch.sum(self.fc(x), dim=1) + self.bias
    

class OneHotFactorizationMachine(torch.nn.Module):

    def __init__(self, field_dimensions, embedding_dimensions, include_linear=False):
        
        super().__init__()
        
        self.embedding = FeaturesEmbedding(field_dimensions, embedding_dimensions)
        
        self.include_linear = include_linear
        
        if self.include_linear:
            
            self.linear = FeaturesLinear(field_dimensions)
        
    
    def predict(self, weights):

        square_of_sum = torch.sum(weights, dim=1) ** 2
        sum_of_squares = torch.sum(weights ** 2, dim=1)
        
        predictions = square_of_sum - sum_of_squares           
        
        predictions = torch.sum(predictions, dim=1, keepdim=True)
        
        predictions = 0.5 * predictions

        # print("square_of_sum:", square_of_sum)    
        # print("sum_of_squares:", sum_of_squares)               
        # print("prediction:", prediction)        
        
        return predictions
    

    def forward(self, fields):  
        
        weights = self.embedding(fields)
        
        predictions = self.predict(weights).squeeze(1)
        
        
        if self.include_linear:
        
            linear_predictions = self.linear(fields).squeeze(1)
            
            predictions += linear_predictions
        
        # print("fields:", fields)    
        # print("weights:", weights)               
        # print("predictions:", predictions.shape, predictions)  
        
        return predictions    

### 2) One Cold Factorization Machine

In [ ]:
class OneColdFactorizationMachine(torch.nn.Module):
    

    def __init__(self, field_dimensions, embedding_dimensions, include_linear=False):
        
        super().__init__()
        
        fields_length = len(field_dimensions)
        
        
        initial_weights = torch.zeros((fields_length, embedding_dimensions))
                              
        self.weights = torch.nn.Parameter(initial_weights)
                                      
        torch.nn.init.xavier_uniform_(self.weights.data)
        
        
        self.include_linear = include_linear
        
        if self.include_linear:
        
            self.linear = torch.nn.Linear(fields_length, 1)
            
        # print("self.weights:", self.weights.shape, self.weights)

                                      
    def forward(self, fields):
               
        masked_weights = fields.unsqueeze(2) * self.weights
        
        square_of_sum = masked_weights.sum(dim=1) ** 2
        
        sum_of_squares = (masked_weights ** 2).sum(dim=1)
        
        predictions = 0.5 * (square_of_sum - sum_of_squares).sum(dim=1)
        
        
        if self.include_linear:
        
            linear_out = self.linear(fields.float()).squeeze()

            predictions += linear_out

        # print("inputs:", inputs.shape, inputs)     
        # print("self.weights:", self.weights.shape, self.weights) 
        # print("masked_weights:", masked_weights.shape, masked_weights)
        # print("square_of_sum:", square_of_sum.shape, square_of_sum)
        # print("sum_of_squares:", sum_of_squares.shape, sum_of_squares)           
        # print("predictions:", predictions.shape, predictions)
        
        return predictions